In [ ]:
#All the following script and comments have been made in accordance to single cell  data ##

#Importing packages # Make sure that you activate correct environment (conda activate scanpy)
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

#writing output file
results_file = "/Users/srivalli/Documents/GitHub/Single-cell-data-analysis/Scanpy/output/heart_immune_scanpy.h5ad"

In [ ]:
#Reading the h5ad file
adata = sc.read_h5ad('/Users/srivalli/Desktop/Heart/hca_heart_immune_download.h5ad')
adata

In [ ]:
#PREPROCESSING#

#Viewing genes that contributes the largest portion in a cell
sc.pl.highest_expr_genes(adata, n_top=20)

#Filtering genes and cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

#Above filtering considers only cells having min 200 genes as a primary criteria and filters the genes which are found in minimum of 3 cells

In [ ]:
# Selecting the group of mitochondrial genes -- GENES OF INTEREST
adata.var["mt"] = adata.var_names.str.startswith("MT-")
adata.var

#Calculating desired metrics 
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace = True)

In [ ]:
#Graphical representation of data

#Violin plots
sc.pl.violin(adata,["n_genes_by_counts", "total_counts", "pct_counts_mt"],jitter=0.4,multi_panel=True)

#Scatter plots
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

In [ ]:
# If we want to slice/Filter the data

#Based on number of genes
adata = adata[adata.obs.n_genes_by_counts < 2500, :]

#Based on percentage of mitochondrial genes
adata = adata[adata.obs.pct_counts_mt < 5, :].copy()

#Normalizing data matrix to have 10,000 reads per cell to make counts comparable
sc.pp.normalize_total(adata, target_sum=1e4)

#Logarithmization of data
sc.pp.log1p(adata)

In [ ]:
#Identifying high variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

In [ ]:
#Saving data as a backup so we wont loose it later

#Saving normalized and logarithmic data which is not corrected as raw

adata.raw = adata

In [ ]:
#Scaling data
#Filtering or selecting columns (genes) based on the highly variable genes identified
adata = adata[:, adata.var.highly_variable]

#Regression of data i.e., Removing batch effects based on criteria of interest
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

#Scaling gene expression to make it comparable across cells
sc.pp.scale(adata, max_value=10)

#To view it
adata

In [ ]:
##PRINCIPAL COMPONENT ANALYSIS##

#Reducing dimensions
sc.tl.pca(adata, svd_solver="arpack")

#To view the principal components
adata.obsm['X_pca']

#Scatter plot for PCA components for visualization 
sc.pl.pca(adata)

#Scatter plot for PCA components for visualization based on coloring of genes
sc.pl.pca(adata,color= "CST3")

In [ ]:
#Number of PCs to be considered for the data
sc.tl.tsne(adata)

#To know the values and count
adata.obsm['X_pca']

In [ ]:
#Estimates of Principal components contribution to the total variance of the data
sc.pl.pca_variance_ratio(adata, log=True)

#Saving results
adata.write(results_file)
adata

In [ ]:
#COMPUTING NEIGHBOUIRHOOD GRAPH#
sc.pp.neighbors(adata)

#Can add n_neighbors and n_pca parameters if we would like to consider making clusters based on given params#

In [ ]:
#EMBEDDING THE NEIGHBOURHOOD GRAPH

#Assinging cells to clusters
sc.tl.louvain(adata)

#Partioning data and identifying relationships between clusters
sc.tl.paga(adata)
sc.pl.paga(adata)

#Data visualization
sc.tl.umap(adata)

#Giving colour codes for better visulauization based on genes
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"])

#For raw data which is logarithmized and normalized but uncorrected
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"], use_raw=False)

In [ ]:
##CLUSTERING NEIGHBORHOOD GRAPH#

#Recommendded method Leiden graph-clustering method
sc.tl.leiden(
    adata,
    resolution=0.9,
    random_state=0,
    n_iterations=2,
    directed=False,
)

sc.pl.umap(adata, color=["leiden"])

#To see the cluster labels
adata.obs['leiden']

In [ ]:
#Saving file
adata.write(results_file)

In [ ]:
#FINDING MARKER GENES#

#Ranking genes using t-test
sc.tl.rank_genes_groups(adata, "leiden", method="t-test")

#Plotting 
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#Plotting 
sc.pl.rank_genes_groups(adata)

In [ ]:
#Ranking genes using wilcoxon method
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")

#Plotting 
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

#Saving data
adata.write(results_file)

In [ ]:
#Using logistic regression
sc.tl.rank_genes_groups(adata, "leiden", method="logreg", max_iter=1000)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#To get list of gene names across clusters
pd.DataFrame(adata.uns["rank_genes_groups"]["names"])

#Can add .head(10) at the end to give output of only 10 hits

In [ ]:
#Getting table with scores and group
adata = sc.read(results_file)
result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
pd.DataFrame(
    {
        group + "_" + key[:1]: result[key][group]
        for group in groups
        for key in ["names", "pvals"]
    }
).head(5)
#low p-value indicates that the gene is likely differentially expressed in that cluster compared to others

In [ ]:
#Comparing between clusters and ranking genes
sc.tl.rank_genes_groups(adata, "leiden", groups=["0"], reference="1", method="wilcoxon")
sc.pl.rank_genes_groups(adata, groups=["0"], n_genes=20)

In [ ]:
#More detailed view of cluster if required
sc.pl.rank_genes_groups_violin(adata, groups="0", n_genes=8)

In [ ]:
#Visualizing genes across clusters
sc.pl.violin(adata, ["CST3"], groupby="leiden")

In [ ]:
#Visualizing gene across groups in dotplot
sc.pl.dotplot(adata, 'CST3', groupby="leiden")

#Visualizing gene across groups in violin plot
sc.pl.stacked_violin(adata, 'CST3', groupby="leiden");

In [ ]:
##SAVING FILE
adata.write(results_file, compression="gzip")